เชื่อม google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


ทำการสร้าง evaluation dataest ภาษาไทยโดยการใช้ LLM เเปลภาษา

In [ ]:
import requests
import time
import json
import os

# 🔑 API Key สำหรับ Together
api_key = "xxxxxxxxxxx"  # 👈 ใส่ Together API Key ของคุณตรงนี้

# 🚀 โหลดไฟล์คำถามพร้อม RAG Recommendations
input_file = '/content/drive/MyDrive/LockLearn/evaluation_datasetv2.json'  # ไฟล์ input
output_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_TH.json'  # ไฟล์ output

# 📂 โหลดข้อมูลจากไฟล์
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"✅ รวมคำถามที่เลือกทั้งหมด: {len(data)}")

# 🚀 โหลดผลลัพธ์ล่าสุด (ถ้ามี)
if os.path.exists(output_file):
    with open(output_file, 'r', encoding='utf-8') as f:
        processed_answers = json.load(f)
        processed_ids = set([item['translated_question'] for item in processed_answers])
else:
    processed_answers = []
    processed_ids = set()

# 🔁 ฟังก์ชันเรียก API
def request_with_retry(prompt, retries=3, delay=5):
    headers = {"Content-Type": "application/json","Authorization": f"Bearer {api_key}"}
    json_data = {
        "model": "meta-llama/llama-4-scout-17b-16e-instruct",
        "messages": [{"role": "user", "content": prompt}]
    }

    for attempt in range(retries):
        try:
            response = requests.post("https://api.together.xyz/v1/chat/completions", headers=headers, json=json_data, timeout=30)
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]
            elif response.status_code == 429:
                print("❌ Rate limit reached. Waiting for 10 seconds...")
                time.sleep(2)  # รอ 10 วินาทีถ้าเจอ rate limit
            else:
                print(f"❌ Error (Status {response.status_code}): {response.text}")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"❌ Connection Error: {e}")
            time.sleep(delay)
    return None

# 🚀 แปลคำถามและคำตอบทั้งหมด
for index, item in enumerate(data, start=1):
    original_question = item['question']
    original_answer = item.get('answer', 'No answer provided')

    if original_question in processed_ids:
        continue

    # สร้าง prompt สำหรับแปลคำถามและคำตอบ
    question_prompt = f"Translate this question into Thai:\n\n{original_question}"
    answer_prompt = f"Translate this answer into Thai:\n\n{original_answer}"

    translated_question = request_with_retry(question_prompt)
    translated_answer = request_with_retry(answer_prompt)

    if translated_question and translated_answer:
        processed_answers.append({
            "translated_question": translated_question.strip(),
            "translated_answer": translated_answer.strip()
        })
        processed_ids.add(original_question)

        # 💾 บันทึกผลลัพธ์ทันที
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(processed_answers, f, ensure_ascii=False, indent=4)

        print(f"✅ คำถามและคำตอบที่ {index} ได้รับการแปลแล้ว")

print("✅ เสร็จสิ้นการแปลทั้งหมด")
